#### Set Up

In [1]:
from tqdm import tqdm
import pandas as pd

from direct_asking import DirectAsking
from utils import load_data

c:\Users\ADMIN\.conda\envs\base-12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test_path = 'data/vihallu-public-test.csv'
output_path = 'data/output/submission_da_qwen3_4b.csv'

test = load_data(test_path)
print(f'Length of test set: {len(test)}')
test.head()

Length of test set: 1000


,id,context,prompt,response,predict_label
0,b709059b-b3b6-4ac2-bb88-2c794e2cc219,"Putin ngày 14 tháng 10 năm 2009, đưa ra đề ngh...",Ý nhĩa cũa viẹc tổ chưc cuôc thi Intervision l...,"Cuộc thi Intervision, dự kiến lấy cảm hứng từ ...",NaN
1,7dc35ef5-c4b7-4538-ab90-627b9cbd896e,"Thông qua những quyết nghị này, Viện nguyên lã...",Quyền lực của Viện nguyên lão đã giảm mạnh khi...,"Sai. Thực tế, Viện nguyên lão vẫn duy trì quyề...",NaN
2,cfdfa010-f61c-4845-91c9-23f79be2b88b,Nơi cư ngụ truyền thống Mông Cổ được gọi là mộ...,Theo quan điểm của ai thì kiến trúc truyền thố...,"Ngoài N. Chultem, một số học giả như nhà sử họ...",NaN
3,31b33c97-2f59-4e72-8707-f47de204d7f9,Là thành phố thủ đô và có vị trí ở khu vực tru...,Hà Nội hiện nay có tuyến đường sắt quốc tế kết...,Hà Nội hiện nay có tuyến đường sắt quốc tế trự...,NaN
4,a2c83a00-e8b7-4236-86ce-5e0104df074a,Từ 1974 đến nay tốc độ phát triển tuy chậm lại...,Câu hỏi gài bẫy: Tiền đề để trở thành một quốc...,Tiền đề để Nhật Bản trở thành quốc gia cho vay...,NaN


In [3]:
direct_asking = DirectAsking(
    model_name='Qwen/Qwen3-4B-Instruct-2507',
    user_prompt_path='prompts/direct_asking_user.txt',
)

Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.27s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


#### Labeling

In [4]:
result = pd.DataFrame(columns=['id', 'predict_label'])

for _, sample in tqdm(test.iterrows(), total=len(test), desc='Processing test'):
    id = sample['id']
    label = direct_asking.predict(
        context = sample['context'],
        prompt = sample['prompt'],
        response = sample['response']
    )
    result = pd.concat([pd.DataFrame([[id, label]], columns=result.columns), result], ignore_index=True)

Processing test: 100%|██████████| 1000/1000 [17:57<00:00,  1.08s/it]


In [5]:
result.to_csv(output_path, index=False)